<a href="https://colab.research.google.com/github/jesseharding24/dune-analytics/blob/main/2023_Crypto_Profit_and_Loss_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About

- This notebook allows you to quickly grab needed info, such as cost basis and profit or loss for each transfer of crypto assets, to simplify your crypto tax filing! 🧾

- All you need to do is to input in your wallet addresses as and run the notebook. 💨

---

## Set up
- install Dune API SDK (run `pip install dune_client`)
- obtain an API key (https://docs.dune.com/api-reference/overview/authentication)

In [ ]:
!pip install dune_client

In [ ]:
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase
import pandas as pd

In [ ]:
dune_api_key = '<your_api_key_here>' # input your API key, to create one follow guide here https://docs.dune.com/api-reference/overview/authentication

In [ ]:
dune = DuneClient(
    api_key=dune_api_key,
    base_url="https://api.dune.com",
    request_timeout=(300) # request will time out after 300 seconds
)

## For EVM wallets

---
### Input our EVM wallets here 👇
_Chains included: Ethereum, Arbitrum, Optimism, Polygon_

In [ ]:
evm_wallets = '<your list of EVM wallets here>'
# Notebook ran with '0xd8da6bf26964af9d7eed9e03e53415d37aa96045, 0xd7029bdea1c17493893aafe29aad69ef892b8ff2' for vitalik.eth, dwr.eth

In [ ]:
query_evm = QueryBase(
    query_id = 3564905, # https://dune.com/queries/3564905
    params=[
        QueryParameter.text_type(name="list_of_wallets", value=evm_wallets),
    ],
)

print("Results available at", query_evm.url())

query_result_evm = dune.run_query(
    query=query_evm,
    performance='large', # optional, default is 'medium' engine size
    ping_frequency=30 # optional, default is 1 (so pinging server every 1 second to check status)
)

Results available at https://dune.com/queries/3564905?list_of_wallets=0xd8da6bf26964af9d7eed9e03e53415d37aa96045%2C+0xd7029bdea1c17493893aafe29aad69ef892b8ff2


In [ ]:
evm_datapoint = query_result_evm.result.metadata.datapoint_count
evm_df = pd.DataFrame(query_result_evm.result.rows)

print("Datapoints downloaded for EVM wallets: ", evm_datapoint)

Datapoints downloaded for EVM wallets:  450744


## For Solana wallets
---
### Input our Solana wallets here 👇
_Not including NFT transfers currently_

In [ ]:
solana_wallets = '<your list of Solana wallets here>'
# Notebook ran with '6by8iRgL41nY5fzFAmoH8dxzWFwmDmYiMbVUq6fpK69d'

In [ ]:
query_solana = QueryBase(
    query_id = 3560535, # https://dune.com/queries/3560535
    params=[
        QueryParameter.text_type(name="list_of_wallets", value=solana_wallets),
    ],
)

print("Results available at", query_solana.url())

query_result_solana = dune.run_query(
    query=query_solana,
    performance='large', # optional, default is 'medium' engine size
    ping_frequency=30 # optional, default is 1 (so pinging server every 1 second to check status)
)

Results available at https://dune.com/queries/3560535?list_of_wallets=6by8iRgL41nY5fzFAmoH8dxzWFwmDmYiMbVUq6fpK69d


In [ ]:
solana_datapoint = query_result_solana.result.metadata.datapoint_count
solana_df = pd.DataFrame(query_result_solana.result.rows)

print("Datapoints downloaded for Solana wallets: ", solana_datapoint)

Datapoints downloaded for Solana wallets:  171


## Getting results
---
- Download line items for EVM wallets and Solana wallets
- Get aggregate profit or loss for the year

In [ ]:
# Calculate Aggregated PnL
aggregated_pnl_evm = evm_df['pnl'].sum()  # Replace 'pnl' with the actual column name for PnL in evm_df
aggregated_pnl_solana = solana_df['profit_usd'].sum()  # Replace 'profit_usd' with the actual column name for profit in USD in solana_df

# Sum of PnLs from both DataFrames
total_aggregated_pnl = aggregated_pnl_evm + aggregated_pnl_solana

print('All wallets PnL for 2023 = ', total_aggregated_pnl)

All wallets PnL for 2023 =  2581130.5766407717


In [ ]:
# Export DataFrames to CSV files
evm_df.to_csv('evm_transactions.csv', index=False)
solana_df.to_csv('solana_transactions.csv', index=False)